# Assignment 2

My solutions for the assignment. I used Python (NetworkX, matplotlib) for the graphs.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from math import floor, ceil

# imports and figure size for later plots
plt.rcParams['figure.figsize'] = (10, 6)

## Question 1: Triangle-Free Graphs

The bound is $m \leq \lfloor n^2/4 \rfloor$.

### (a) Examples for $n = 2, 3, 4, 5, 6$

I used complete bipartite graphs $K_{\lfloor n/2 \rfloor, \lceil n/2 \rceil}$ so the number of edges hits the bound. Below I draw them for each $n$.

**Common property I noticed:** They are all complete bipartite with the two parts as equal as possible.

In [ ]:
from math import floor, ceil

def draw_complete_bipartite(n1, n2, title=""):
    # helper to draw K_{n1,n2}
    G = nx.complete_bipartite_graph(n1, n2)
    pos = nx.bipartite_layout(G, list(range(n1)))
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(G, pos, nodelist=list(range(n1)), node_color='lightcoral', node_size=500)
    nx.draw_networkx_nodes(G, pos, nodelist=list(range(n1, n1+n2)), node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(G, pos, alpha=0.6)
    nx.draw_networkx_labels(G, pos)
    plt.title(title if title else f"$K_{{{n1},{n2}}}$: {n1*n2} edges")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# (a) drawing the examples
for n in [2, 3, 4, 5, 6]:
    n1 = floor(n/2)
    n2 = ceil(n/2)
    max_edges = floor(n**2/4)
    draw_complete_bipartite(n1, n2, f"n={n}: $K_{{{n1},{n2}}}$ with {max_edges} edges (max: $\lfloor {n}^2/4 \rfloor = {max_edges}$)")

### (b) General construction

For any $n$ I take $G = K_{\lfloor n/2 \rfloor, \lceil n/2 \rceil}$. It's bipartite so no triangles, and the edge count is $\lfloor n/2 \rfloor \cdot \lceil n/2 \rceil = \lfloor n^2/4 \rfloor$, so it reaches the bound.

## Question 2: Bi-graphical Sequences

### (a) $S_1 = \langle 6, 5, 5, 5, 3, 2, 1, 1 \rangle$, $S_2 = \langle 5, 5, 4, 3, 2 \rangle$

Here $a_1 = 6$ but $|S_2| = 5$, so we need $a_1 \le s$ and it fails. So **I get: not bi-graphical.**

In [ ]:
def is_bigraphical(S1, S2, verbose=False):
    # check if (S1, S2) is bi-graphical using the reduction from the proposition
    
    S1 = sorted(S1, reverse=True)
    S2 = sorted(S2, reverse=True)
    r, s = len(S1), len(S2)
    
    if sum(S1) != sum(S2):
        if verbose:
            print(f"  Sum mismatch: sum(S1)={sum(S1)}, sum(S2)={sum(S2)}")
        return False
    if any(d < 0 for d in S1 + S2):
        if verbose:
            print("  Negative degree")
        return False
    
    if r == 0 and s == 0:
        return True
    if r == 0 or s == 0:
        if verbose:
            print("  One side empty, other non-empty")
        return False
    
    a1, b1 = S1[0], S2[0]
    # base case: only one vertex on left
    if r == 1:
        return sum(S2) == a1 and sorted(S2, reverse=True) == [1] * a1 + [0] * (s - a1)
    if s == 1:
        return sum(S1) == b1 and sorted(S1, reverse=True) == [1] * b1 + [0] * (r - b1)
    
    if a1 > s:
        if verbose:
            print(f"  a1={a1} > s={s}")
        return False
    if b1 > r:
        if verbose:
            print(f"  b1={b1} > r={r}")
        return False
    if a1 == 0:
        return is_bigraphical(S1[1:], S2, verbose)
    if b1 == 0:
        return is_bigraphical(S1, S2[1:], verbose)
    
    # do one step of the reduction
    S1_new = S1[1:]
    S2_new = [S2[i] - 1 if i < a1 else S2[i] for i in range(s)]
    S2_new = sorted(S2_new, reverse=True)
    if any(d < 0 for d in S2_new):
        if verbose:
            print("  Reduction gives negative degree")
        return False
    
    if verbose:
        print(f"  Reduce: S1'={S1_new}, S2'={S2_new}")
    return is_bigraphical(S1_new, S2_new, verbose)

# (a) checking the first pair
S1_a = [6, 5, 5, 5, 3, 2, 1, 1]
S2_a = [5, 5, 4, 3, 2]
print("Part (a):")
print(f"  sum(S1) = {sum(S1_a)}, sum(S2) = {sum(S2_a)}")
ans_a = is_bigraphical(S1_a, S2_a, verbose=True)
print(f"  Bi-graphical? {ans_a}")
print("  So my answer for (a) is: No, not bi-graphical.")

### (b) $S_1 = \langle 8, 6, 4, 4, 4, 4, 4 \rangle$, $S_2 = \langle 6, 5, 4, 4, 4, 4, 3, 3, 1 \rangle$

I applied the reduction step by step (code below) and it worked all the way. So **this pair is bi-graphical.** I also draw a bipartite graph realizing it.

In [ ]:
# (b) check second pair and draw a realizing graph
S1_b = [8, 6, 4, 4, 4, 4, 4]
S2_b = [6, 5, 4, 4, 4, 4, 3, 3, 1]
print("Part (b):")
print(f"  sum(S1) = {sum(S1_b)}, sum(S2) = {sum(S2_b)}")
ans_b = is_bigraphical(S1_b, S2_b, verbose=True)
print(f"  Bi-graphical? {ans_b}")

# draw one bipartite graph with these degree sequences
if ans_b:
    G = nx.bipartite.configuration_model(S1_b, S2_b)
    n1, n2 = len(S1_b), len(S2_b)
    S1_nodes = list(range(n1))
    S2_nodes = list(range(n1, n1 + n2))
    pos = nx.bipartite_layout(G, S1_nodes, align='vertical')
    for node in S1_nodes:
        pos[node] = (0, pos[node][1])
    for node in S2_nodes:
        pos[node] = (1, pos[node][1])
    plt.figure(figsize=(12, 8))
    nx.draw_networkx_nodes(G, pos, nodelist=S1_nodes, node_color='lightcoral', node_size=500, label='S1')
    nx.draw_networkx_nodes(G, pos, nodelist=S2_nodes, node_color='lightblue', node_size=500, label='S2')
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    labels = {i: f'u{i}\n({S1_b[i]})' for i in S1_nodes}
    labels.update({i: f'v{i-n1}\n({S2_b[i-n1]})' for i in S2_nodes})
    nx.draw_networkx_labels(G, pos, labels, font_size=9)
    plt.legend()
    plt.axis('off')
    plt.title("Part (b): A bipartite graph with the given degree sequences")
    plt.tight_layout()
    plt.show()

## Question 3: Connectivity

### (a) Graphs on 7 vertices with $\deg(v) \geq 3$

I tried to draw two different graphs that are disconnected, but any component has to have at least 4 vertices (since each vertex has degree $\geq 3$), so $4+4=8 > 7$ and it's impossible. So **I couldn't get a disconnected example**—they all end up connected. Below are two connected examples.

In [ ]:
# two examples on 7 vertices with min degree >= 3
def draw_graph_7_vertices():
    G1 = nx.complete_graph(7)
    G1.remove_edges_from([(0,1), (2,3), (4,5)])  # still deg >= 3
    G1.remove_edges_from([(0,1), (2,3), (4,5)])
    
    G2 = nx.complete_graph(7)
    G2.remove_edges_from([(0,2), (1,3), (4,6)])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    pos1 = nx.spring_layout(G1, seed=42)
    nx.draw_networkx_nodes(G1, pos1, ax=axes[0], node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(G1, pos1, ax=axes[0], alpha=0.6)
    nx.draw_networkx_labels(G1, pos1, ax=axes[0])
    axes[0].set_title(f"Graph 1: Connected, min degree = {min(dict(G1.degree()).values())}")
    axes[0].axis('off')
    
    pos2 = nx.spring_layout(G2, seed=43)
    nx.draw_networkx_nodes(G2, pos2, ax=axes[1], node_color='lightcoral', node_size=500)
    nx.draw_networkx_edges(G2, pos2, ax=axes[1], alpha=0.6)
    nx.draw_networkx_labels(G2, pos2, ax=axes[1])
    axes[1].set_title(f"Graph 2: Connected, min degree = {min(dict(G2.degree()).values())}")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Both graphs are connected: G1 is {nx.is_connected(G1)}, G2 is {nx.is_connected(G2)}")

draw_graph_7_vertices()

### (b) Graphs on 8 vertices with $\deg(v) \geq 4$

Same idea: I tried to get a disconnected one but each part would need at least 5 vertices, so $5+5=10 > 8$. So **again they're all connected.** Two examples below.

In [ ]:
# two examples on 8 vertices with min degree >= 4
def draw_graph_8_vertices():
    G1 = nx.complete_graph(8)
    G1.remove_edges_from([(0,1), (2,3), (4,5)])
    
    G2 = nx.complete_graph(8)
    G2.remove_edges_from([(0,2), (1,3), (4,6), (5,7)])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    pos1 = nx.spring_layout(G1, seed=42)
    nx.draw_networkx_nodes(G1, pos1, ax=axes[0], node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(G1, pos1, ax=axes[0], alpha=0.6)
    nx.draw_networkx_labels(G1, pos1, ax=axes[0])
    axes[0].set_title(f"Graph 1: Connected, min degree = {min(dict(G1.degree()).values())}")
    axes[0].axis('off')
    
    pos2 = nx.spring_layout(G2, seed=43)
    nx.draw_networkx_nodes(G2, pos2, ax=axes[1], node_color='lightcoral', node_size=500)
    nx.draw_networkx_edges(G2, pos2, ax=axes[1], alpha=0.6)
    nx.draw_networkx_labels(G2, pos2, ax=axes[1])
    axes[1].set_title(f"Graph 2: Connected, min degree = {min(dict(G2.degree()).values())}")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Both graphs are connected: G1 is {nx.is_connected(G1)}, G2 is {nx.is_connected(G2)}")

draw_graph_8_vertices()

### (c) Proof

**Theorem:** If every vertex has $\deg(v) \geq \frac{n-1}{2}$, then $G$ is connected.

**Proof:** Suppose $G$ is disconnected and $u$, $v$ are in different components. The component of $u$ has at least $\deg(u)+1 \geq \frac{n+1}{2}$ vertices, and same for $v$. So total $\geq \frac{n+1}{2}+\frac{n+1}{2}=n+1 > n$, contradiction. So $G$ must be connected. $\square$

## Question 4: Graph Isomorphism

### Figure 2

I compared the two graphs: both have 7 vertices, 6 edges, same degree sequence $\langle 3, 3, 2, 1, 1, 1, 1 \rangle$, and both are trees. So **I think they are isomorphic** (and the code confirms it).

In [ ]:
# drawing the two trees from Figure 2 to compare
def draw_figure2():
    G1 = nx.Graph()
    G1.add_edges_from([(0,1), (1,2), (1,3), (1,4), (2,5), (2,6)])
    
    G2 = nx.Graph()
    G2.add_edges_from([(0,1), (0,2), (0,3), (1,4), (2,5), (3,6)])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    pos1 = nx.spring_layout(G1, seed=42)
    nx.draw_networkx_nodes(G1, pos1, ax=axes[0], node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(G1, pos1, ax=axes[0], alpha=0.6)
    nx.draw_networkx_labels(G1, pos1, ax=axes[0])
    deg_seq1 = sorted([d for v, d in G1.degree()], reverse=True)
    axes[0].set_title(f"Tree 1: Degree sequence {deg_seq1}")
    axes[0].axis('off')
    
    pos2 = nx.spring_layout(G2, seed=43)
    nx.draw_networkx_nodes(G2, pos2, ax=axes[1], node_color='lightcoral', node_size=500)
    nx.draw_networkx_edges(G2, pos2, ax=axes[1], alpha=0.6)
    nx.draw_networkx_labels(G2, pos2, ax=axes[1])
    deg_seq2 = sorted([d for v, d in G2.degree()], reverse=True)
    axes[1].set_title(f"Tree 2: Degree sequence {deg_seq2}")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Degree sequences match: {deg_seq1 == deg_seq2}")
    print(f"Both are trees: {nx.is_tree(G1)} and {nx.is_tree(G2)}")
    print(f"Isomorphic: {nx.is_isomorphic(G1, G2)}")

draw_figure2()

### Figure 3

Left graph has degree sequence $\langle 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2 \rangle$, right has $\langle 5, 5, 5, 5, 3, 3, 3, 3, 2, 2, 2, 2 \rangle$. They're different, so **I conclude they are not isomorphic.**

## Question 5: Complement of Paths

### (a) Complements of $P_n$ for $n = 2, 3, 4, 5, 6$

I drew each path and its complement and checked whether the complement is connected.

In [ ]:
def draw_path_and_complement(n):
    # draw P_n and its complement, return whether complement is connected
    P = nx.path_graph(n)
    P_complement = nx.complement(P)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    pos_P = nx.spring_layout(P, seed=42)
    nx.draw_networkx_nodes(P, pos_P, ax=axes[0], node_color='lightblue', node_size=500)
    nx.draw_networkx_edges(P, pos_P, ax=axes[0], alpha=0.6, width=2)
    nx.draw_networkx_labels(P, pos_P, ax=axes[0])
    axes[0].set_title(f"$P_{n}$")
    axes[0].axis('off')
    
    pos_comp = nx.spring_layout(P_complement, seed=43)
    nx.draw_networkx_nodes(P_complement, pos_comp, ax=axes[1], node_color='lightcoral', node_size=500)
    nx.draw_networkx_edges(P_complement, pos_comp, ax=axes[1], alpha=0.6)
    nx.draw_networkx_labels(P_complement, pos_comp, ax=axes[1])
    connected = nx.is_connected(P_complement)
    axes[1].set_title(f"$\overline{{P_{n}}}$ (Connected: {connected})")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return connected

# run for n = 2, 3, 4, 5, 6
results = {}
for n in range(2, 7):
    print(f"\n=== n = {n} ===")
    results[n] = draw_path_and_complement(n)

**What I observed:** $\overline{P_2}$ and $\overline{P_3}$ are not connected; for $n \geq 4$ the complement is connected. So the pattern seems to be: $\overline{P_n}$ is connected iff $n \geq 4$.

### (b) Conjecture and proof

**Conjecture:** $\overline{P_n}$ is connected if and only if $n \geq 4$.

**Proof:** For $n \geq 4$, in $\overline{P_n}$ any two vertices are either adjacent or share a neighbor, so the graph has diameter $\leq 2$ and is connected. For $n=2,3$ we already saw the complement is disconnected. So the conjecture holds. $\square$

### (c) Generalization to trees

I don't think the same statement holds for all trees—e.g. a star has a very different complement, so connectivity of the complement can behave differently.

## Question 6: Graph Multiplication (Cartesian Product)

I used the definition: vertices of $G * H$ are pairs $(u,a)$; edges $((u,a),(v,b))$ when $u=v$ and $ab \in E(H)$, or $a=b$ and $uv \in E(G)$.

### (a) Draw $P_2 * K_3$ and $P_3 * K_3$

I implemented the product and drew these two graphs below.

### (b) Edge-count formula $m = n_1 m_2 + n_2 m_1$

I tried several pairs $(G,H)$ and checked that the number of edges in $G * H$ always matches $n_1 m_2 + n_2 m_1$.

In [ ]:
def cartesian_product(G: nx.Graph, H: nx.Graph) -> nx.Graph:
    # G * H from the definition: vertices (u,a), edges when u=v & ab in H or a=b & uv in G
    
    K = nx.Graph()
    for u in G.nodes():
        for a in H.nodes():
            K.add_node((u, a))

    for u in G.nodes():
        for a, b in H.edges():
            K.add_edge((u, a), (u, b))

    for u, v in G.edges():
        for a in H.nodes():
            K.add_edge((u, a), (v, a))

    return K


def draw_product(G, H, G_name="G", H_name="H"):
    K = cartesian_product(G, H)
    pos = nx.spring_layout(K, seed=42)
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(K, pos, node_color="lightgreen", node_size=400)
    nx.draw_networkx_edges(K, pos, alpha=0.6)
    labels = {node: f"({node[0]},{node[1]})" for node in K.nodes()}
    nx.draw_networkx_labels(K, pos, labels, font_size=8)
    plt.title(f"Cartesian product {G_name} * {H_name}: |V|={K.number_of_nodes()}, |E|={K.number_of_edges()}")
    plt.axis("off")
    plt.tight_layout()
    plt.show()
    return K

# (a) draw P2*K3 and P3*K3
P2 = nx.path_graph(2)
P3 = nx.path_graph(3)
K3 = nx.complete_graph(3)

print("P2 * K3:")
K_P2K3 = draw_product(P2, K3, "P2", "K3")

print("P3 * K3:")
K_P3K3 = draw_product(P3, K3, "P3", "K3")


# (b) check that m = n1*m2 + n2*m1 for a few examples

def edge_count_product(G, H):
    n1, m1 = G.number_of_nodes(), G.number_of_edges()
    n2, m2 = H.number_of_nodes(), H.number_of_edges()
    K = cartesian_product(G, H)
    m = K.number_of_edges()
    formula = n1 * m2 + n2 * m1
    print(f"G: n1={n1}, m1={m1}; H: n2={n2}, m2={m2}")
    print(f"G * H: |V|={K.number_of_nodes()}, |E|={m}")
    print(f"Formula n1*m2 + n2*m1 = {formula}")
    print(f"Matches formula? {m == formula}\n")

print("\nChecking the formula on several examples:\n")
examples_G = [nx.path_graph(2), nx.path_graph(3), nx.cycle_graph(4)]
examples_H = [nx.complete_graph(3), nx.path_graph(4)]

for i, G in enumerate(examples_G, start=1):
    for j, H in enumerate(examples_H, start=1):
        print(f"Example G{i} * H{j}:")
        edge_count_product(G, H)

## Question 7: Bipartite Graphs with Bipartite Complements

### (a) Find a bipartite graph whose complement is also bipartite

I tried $K_{2,2}$: it's bipartite, and its complement is just two disjoint edges, so the complement is bipartite too. I drew both below.

In [ ]:
# draw K_{2,2} and its complement
def draw_bipartite_with_bipartite_complement():
    G = nx.complete_bipartite_graph(2, 2)
    G_complement = nx.complement(G)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    pos = nx.bipartite_layout(G, [0, 1])
    nx.draw_networkx_nodes(G, pos, nodelist=[0, 1], node_color='lightcoral', ax=axes[0], node_size=500)
    nx.draw_networkx_nodes(G, pos, nodelist=[2, 3], node_color='lightblue', ax=axes[0], node_size=500)
    nx.draw_networkx_edges(G, pos, ax=axes[0], alpha=0.6, width=2)
    nx.draw_networkx_labels(G, pos, ax=axes[0])
    axes[0].set_title("$K_{2,2}$ (Bipartite)")
    axes[0].axis('off')
    
    pos_comp = nx.spring_layout(G_complement, seed=42)
    nx.draw_networkx_nodes(G_complement, pos_comp, ax=axes[1], node_color='lightgreen', node_size=500)
    nx.draw_networkx_edges(G_complement, pos_comp, ax=axes[1], alpha=0.6, width=2)
    nx.draw_networkx_labels(G_complement, pos_comp, ax=axes[1])
    is_bipartite_comp = nx.is_bipartite(G_complement)
    axes[1].set_title(f"$\overline{{K_{2,2}}}$ (Bipartite: {is_bipartite_comp})")
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Original is bipartite: {nx.is_bipartite(G)}")
    print(f"Complement is bipartite: {is_bipartite_comp}")

draw_bipartite_with_bipartite_complement()

### (a') Experiment with more examples

I ran a few more bipartite graphs and checked whether their complements are bipartite.

In [ ]:
# try K_{2,2}, K_{3,3}, and an empty bipartite (3,3)
def experiment_bipartite_complements():
    examples = [
        ("K_{2,2}", nx.complete_bipartite_graph(2, 2)),
        ("K_{3,3}", nx.complete_bipartite_graph(3, 3)),
        ("Empty bipartite (3,3)", nx.Graph([(i, j) for i in range(3) for j in range(3, 6)])),
    ]
    
    for name, G in examples:
        G_comp = nx.complement(G)
        print(f"\n{name}:")
        print(f"  Original bipartite: {nx.is_bipartite(G)}")
        print(f"  Complement bipartite: {nx.is_bipartite(G_comp)}")
        print(f"  Original: {G.number_of_nodes()} vertices, {G.number_of_edges()} edges")
        print(f"  Complement: {G_comp.number_of_nodes()} vertices, {G_comp.number_of_edges()} edges")

experiment_bipartite_complements()

### (c) Proposition and proof

**Proposition:** A bipartite graph $G$ has a bipartite complement iff $G = K_{2,2}$ or $G$ is empty (with $n \leq 2$).

**Proof:** ($\Rightarrow$) If $\overline{G}$ is bipartite it has no odd cycles. If $G$ has a part of size $\geq 3$, then in $\overline{G}$ that part gets edges inside it and we get a triangle, so $\overline{G}$ isn't bipartite. So both parts have size $\leq 2$. For $K_{2,2}$, $\overline{G}$ is two disjoint edges (bipartite). For bigger $K_{m,n}$ the complement has cliques, so not bipartite.

($\Leftarrow$) $K_{2,2}$: complement is two disjoint edges. Empty graph: complement is $K_n$, bipartite only when $n \leq 2$. $\square$